<a href="https://colab.research.google.com/github/ChaejinE/MyPytorch/blob/main/PyTorch_Tips_Details/o_Modules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
import torch.nn as nn

# Overview
- Module : 여러 개의 작은 블록으로 구성된 큰 블록이 있을 때
- Sequential : 레이어에서 작은 블록을 만들고 싶을 때
- ModuleList : 일부 레이어 또는 빌딩 블록을 반복하면서 어떤 작업을 해야할 때
- ModuleDict : 모델의 일부 블록을 매개 변수화 해야하는 경우 (?? activation의 기능이 예시랜다.)

# Module
- 가장 기본이 되는 block 단위
- Conv -> BatchNorm -> ReLU 가 이어져서 사용되지만, 함수처럼 사용하지 못하는 것은 다소 비효율적인 것으로 보인다.

In [3]:
import torch.nn.functional as F

class CNNClassifier(nn.Module):
  def __init__(self, in_c, n_classes):
    super().__init__()
    self.conv1 = nn.Conv2d(in_c, 32, kernel_size=3, stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(32)

    self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(32)

    self.fc1 = nn.Linear(32 * 28 * 28, 1024)
    self.fc2 = nn.Linear(1024, n_classes)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = F.relu(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = F.relu(x)

    x = x.view(x.size(0), -1) # batch별로 flatten

    x = self.fc1(x)
    x = F.sigmoid(x)
    x = self.fc2(x)

    return x

# Sequential
- 마치 컨테이너 처럼 Module을 담는 역할을 한다.
- Sequential에 쌓은 순서대로 Module은 실행되고, 같은 Sequential에 쌓은 모듈은 한 단위처럼 실행된다.
- 코드가 간결해진다.
- 하지만 conv_block1, conv_block2 로 코드가 중복된다. 중복되는 코드를 함수로 빼면 더 간결하게 쓸 수 있다.

In [14]:
import torch.nn.functional as F

class CNNClassifier(nn.Module):
  def __init__(self, in_c, n_classes):
    super().__init__()
    self.conv_block1 = nn.Sequential(
        nn.Conv2d(in_c, 32, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU()
    )

    self.conv_block2 = nn.Sequential(
        nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU()
    )

    self.decoder = nn.Sequential(
        nn.Linear(32 * 28 * 28, 1024),
        nn.Sigmoid(),
        nn.Linear(1024, n_classes)
    )

    self.fc1 = nn.Linear(32 * 28 * 28, 1024)
    self.fc2 = nn.Linear(1024, n_classes)

  def forward(self, x):
    x = self.conv_block1(x)
    x = self.conv_bolck2(x)

    x = x.view(x.size(0), -1)

    x = self.decoder(x)

    return x

In [16]:
import torch.nn.functional as F

def conv_block(in_f, out_f, *args, **kwargs):
  return nn.Sequential(
        nn.Conv2d(in_f, out_f, *arg, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU()
  )

class CNNClassifier(nn.Module):
  def __init__(self, in_c, n_classes):
    super().__init__()
    self.conv_block1 = conv_block(in_c, 32, kernel_size=3, padding=1)
    self.conv_block2 = conv_block(32, 64, kernel_size=3, padding=1)

    self.decoder = nn.Sequential(
        nn.Linear(32 * 28 * 28, 1024),
        nn.Sigmoid(),
        nn.Linear(1024, n_classes)
    )

    self.fc1 = nn.Linear(32 * 28 * 28, 1024)
    self.fc2 = nn.Linear(1024, n_classes)

  def forward(self, x):
    x = self.conv_block1(x)
    x = self.conv_bolck2(x)

    x = x.view(x.size(0), -1)

    x = self.decoder(x)

    return x

- 더 간결하게 만든 것이지만 encoder 부분이 계속 늘어난다면 좋은 방법은 아니다.


In [18]:
import torch.nn.functional as F

def conv_block(in_f, out_f, *args, **kwargs):
  return nn.Sequential(
        nn.Conv2d(in_f, out_f, *arg, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU()
  )

class CNNClassifier(nn.Module):
  def __init__(self, in_c, n_classes):
    super().__init__()
    self.encoder = nn.Sequential(
        conv_block(in_c, 32, kernel_size=3, padding=1),
        conv_block(32, 64, kernel_size=3, padding=1)
    )

    self.decoder = nn.Sequential(
        nn.Linear(32 * 28 * 28, 1024),
        nn.Sigmoid(),
        nn.Linear(1024, n_classes)
    )

    self.fc1 = nn.Linear(32 * 28 * 28, 1024)
    self.fc2 = nn.Linear(1024, n_classes)

  def forward(self, x):
    x = self.encoder(x)
    x = x.view(x.size(0), -1)
    x = self.decoder(x)

    return x

- 예를들어
```python
self.encoder = nn.Sequential(
            conv_block(in_c, 32, kernel_size=3, padding=1),
            conv_block(32, 64, kernel_size=3, padding=1),
            conv_block(64, 128, kernel_size=3, padding=1),
            conv_block(128, 256, kernel_size=3, padding=1),

        )
```
- 이런 경우 반복문을 이용해 코드를 간결하게 작성할 수 있다. (input, output의 Channel 수)
- Input과 output의 channel 수는 list를 이용해 정의해 두는 방법을 많이 사용한다.
  - 핵심은 **반복문을 사용하되 channel의 크기를 미리 저장해 두고 사용하면 된다는 것이다.**

In [19]:
import torch.nn.functional as F

def conv_block(in_f, out_f, *args, **kwargs):
  return nn.Sequential(
        nn.Conv2d(in_f, out_f, *arg, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU()
  )

class CNNClassifier(nn.Module):
  def __init__(self, in_c, n_classes):
    super().__init__()
    self.enc_sizes = [in_c, 32, 64]
    # N 번째 블럭의 output channel의 수가 N+1 번째 block의 input channel 수가 된다
    # 이를 이용해 리스트를 교차로 접근한다.
    self.conv_blocks = [conv_block(in_f, out_f, kernel_size=3, padding=1)
                        for in_f, out_f in zip(self.enc_sizes, self.enc_sizes[1:])]

    # *연산자를 리스트와 같이 사용하면 편하게 사용할 수 있다.
    # container unpacking method
    self.encoder = nn.Sequential(*conv_blocks)

    self.decoder = nn.Sequential(
        nn.Linear(32 * 28 * 28, 1024),
        nn.Sigmoid(),
        nn.Linear(1024, n_classes)
    )

    self.fc1 = nn.Linear(32 * 28 * 28, 1024)
    self.fc2 = nn.Linear(1024, n_classes)

  def forward(self, x):
    x = self.encoder(x)
    x = x.view(x.size(0), -1)
    x = self.decoder(x)

    return x

In [21]:
a = [1, 2, 3]
b = a[1:]

In [23]:
list(zip(a, b))

[(1, 2), (2, 3)]

In [24]:
a = [1,2,3,4,5]
b = [10, *a]
b

[10, 1, 2, 3, 4, 5]

In [26]:
c = [10, a]
c

[10, [1, 2, 3, 4, 5]]

- 더 간결하게 정리해본 코드다.

In [27]:
import torch.nn.functional as F

def conv_block(in_f, out_f, *args, **kwargs):
  return nn.Sequential(
        nn.Conv2d(in_f, out_f, *arg, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU()
  )

def dec_block(in_f, out_f):
  return nn.Sequential(
      nn.Linear(in_f, out_f),
      nn.Sigmoid()
  )

class CNNClassifier(nn.Module):
  def __init__(self, in_c, enc_sizes, dec_size, n_classes):
    super().__init__()
    self.enc_sizes = [in_c, *enc_sizes]
    self.dec_sizes = [32 * 28 * 28, *dec_sizes]

    self.conv_blocks = [conv_block(in_f, out_f, kernel_size=3, padding=1)
                        for in_f, out_f in zip(self.enc_sizes, self.enc_sizes[1:])]
    self.encoder = nn.Sequential(*conv_blocks)

    self.dec_blocks = [dec_block(in_f, out_f)
                        for in_f, out_f in zip(self.dec_sizes, self.dec_sizes[1:])]
    self.decoder = nn.Sequential(*dec_blocks)
    
    self.last = nn.Linear(self.dec_sizes[-1], n_classes)

  def forward(self, x):
    x = self.encoder(x)
    x = x.view(x.size(0), -1)
    x = self.decoder(x)

    return x

# ModuleList
- Module 리스트 형태로 담을 때 사용한다.
  - Sequential과 동일하게 저장한 모듈을 차례대로 접근하며 실행
  - 차이는 내부적으로 forward를 사용하냐 안하냐이다.
- Sequential은 내부 레이어에 접근하여 어떤 작업을 하는데에 어려움이 있다.
- 반면, ModuleList는 리스트 형태로 각 Module에 접근해서 사용할 수 있다.
  - forward 함수에서 for문을 통해 iterate 하며 Module들을 실행한다.

In [30]:
class MyModule(nn.Module):
  def __init__(self, sizes):
    super().__init__()
    self.layers = nn.ModuleList([nn.Linear(in_f, out_f) for in_f, out_f in zip(sizes, sizes[1:])])
    self.trace = []

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
      self.trace.append(x)
    return x

model = MyModule([1, 16, 32])
model(torch.rand((4, 1)))
[print(trace.shape) for trace in model.trace]

torch.Size([4, 16])
torch.Size([4, 32])


[None, None]

# ModuleDict
- 모듈을 딕셔너리 형태로 사용할 수 있다.

In [34]:
def conv_block(in_f, out_f, activation="relu", *args, **kwargs):
  activations = nn.ModuleDict([
                               ["lrelu", nn.LeakyReLU()],
                               ['relu', nn.ReLU()]
  ])

  return nn.Sequential(
      nn.Conv2d(in_f, out_f, *args, **kwargs),
      nn.BatchNorm2d(out_f),
      activations[activation]
  )

print(conv_block(1, 32, 'lrelu', kernel_size=3, padding=1))
print(conv_block(1, 32, 'relu', kernel_size=3, padding=1))

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.01)
)
Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)
